In [1]:
# don't ever remove
%env QT_QPA_PLATFORM=wayland

env: QT_QPA_PLATFORM=wayland


In [2]:
import random
import gym
import gym_snake
import numpy as np
from matplotlib import pyplot as plt
import cv2
from gym_snake.envs.constants import Direction4, Action4

register(
    id='Snake--4x4-DeadApple--v0',
    entry_point='gym_snake.envs:Snake__4x4_DeadApple_'
)
register(
    id='Snake-Hex-4x4-DeadApple--v0',
    entry_point='gym_snake.envs:Snake_Hex_4x4_DeadApple_'
)
register(
    id='Snake--8x8-DeadApple--v0',
    entry_point='gym_snake.envs:Snake__8x8_DeadApple_'
)
register(
    id='Snake-Hex-8x8-DeadApple--v0',
    entry_point='gym_snake.envs:Snake_Hex_8x8_DeadApple_'
)
register(
    id='Snake--16x16-DeadApple--v0',
    entry_point='gym_snake.envs:Snake__16x16_DeadApple_'
)
register(
    id='Snake-Hex-16x16-DeadApple--v0',
    entry_point='gym_snake.envs:Snake_Hex_16x16_DeadApple_'
)
register(
    id='Snake--4x4---v0',
    entry_point='gym_snake.envs:Snake__4x4__'
)
register(
    id='Snake-Hex-4x4---v0',
    entry_point='gym_snake.envs:Snake_Hex_4x4__'
)
register(
    id='Snake--8x8---v0',
    entry_point='gym_snake.envs:Snake__8x8__'
)
register(
    id='Snake-Hex-8x8---v0',
    entry_point='gym_snake.envs:Snake_Hex_8x8__'
)
register(
    

In [3]:
%matplotlib tk

In [4]:
env = gym.make("Snake-8x8-v0")

/home/anetakahleova/miniconda3/envs/Snake/lib/python3.9/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


In [5]:
obs = env.reset()

In [6]:
obs

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0, 255],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0, 255,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0, 255,   0],
        [  0,   0,   0],
        [255,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0, 255,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0, 255,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0

In [7]:
plt.ion()
plt.show()

def render(obs):
    plt.imshow(obs)
    plt.draw()
    plt.pause(0.001)
render(obs)

In [8]:
for i in range(1000):
    #env.render
    render(obs)
    action = random.choice([0, 1, 2])

    obs, reward, done, info = env.step(action)
    print(info)

    if done:
        obs = env.reset()

        for y in range(obs.shape[1]):
            for x in range(obs.shape[0]):
                pixel = obs[y][x]
                if pixel[2] == 255:
                    hlava = [y, x]
                if pixel[1] == 255:
                    telo = [y, x]

        if hlava[0] == telo[0]:#stejny radek
            if hlava[1] > telo[1]:
                look_dir = Direction4.east
            else:
                look_dir = Direction4.west
        else:
            if hlava[0] > telo[0]:
                look_dir = Direction4.south
            else:
                look_dir = Direction4.north



        info =

env.close()

SyntaxError: invalid syntax (3814168807.py, line 33)

In [ ]:
# Inspired by Tutorial: https://www.youtube.com/watch?v=dpBKz1wxE_c
# Using Deep Q neural Network on CartPole environment
# Edited
# by Patrik Vácal

from collections import deque
from time import sleep
import gym
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.optimizer_v2.adam import Adam

In [ ]:
# To fix tf2 speed - https://github.com/tensorflow/tensorflow/issues/33024
tf.compat.v1.disable_eager_execution()


In [ ]:
### Prepare algorithm ###
# Q Network - we use one layer so it is not deep
class DQNAgent:
    def __init__(self, env, gamma=0.97, epsilon=1.0, epsilon_decay=0.98, min_epsilon=0.01, learning_rate=0.01,
                 state_dim=None, action_size=None):
        self.gamma = gamma  # discount rate
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.learning_rate = learning_rate

        self.state_dim = state_dim or env.observation_space.shape
        self.action_size = action_size or env.action_space.n

        self.replay_buffer = deque(maxlen=2000)
        self.model = self.build_model()

    def build_model(self):
        model = Sequential([
            Dense(200, input_shape=self.state_dim, activation="relu"),
            Dense(100, activation="relu"),
            Dense(5, activation="relu"),
            Dense(self.action_size, activation=None)
        ])
        model.compile(loss="mean_squared_error", optimizer=Adam(lr=self.learning_rate))
        model.summary(line_length=120)
        return model

    def get_action(self, state, skip_random=False):
        if not skip_random and random.random() < self.epsilon:
            action_random = random.choice(range(self.action_size))  # Get random action
            return action_random

        else:
            q_state = self.model.predict(np.array([state]))  # Select Q values for current state from Q table
            action_greedy = np.argmax(q_state)               # Select action with highest Q value
            return action_greedy

    def add_experience(self, experience):
        self.replay_buffer.append(experience)

    def train_network(self, batch_size=50):
        # Do not train if not enough experience
        if len(self.replay_buffer) < batch_size:
            return

        samples = random.choices(self.replay_buffer, k=batch_size)
        state, action, next_state, reward, done = (list(col) for col in zip(*samples))  # Make list of states, list of actions, ...

        q_current = self.model.predict(np.array(state))            # Get Q values for all possible current actions

        q_next = self.model.predict(np.array(next_state))          # Get Q values for all possible next actions
        q_next[done] = 0                                                   # Set all future(next) Q values to 0 if this step is the last one
        q_target = reward + (self.gamma * np.max(q_next, axis=1))  # Calculate target Q = reward + max next Q value

        for i in range(batch_size):
            q_update = q_target[i] - q_current[i][action[i]]        # = how much to move Q value in table from current value
            q_current[i][action[i]] += self.learning_rate * q_update   # Update Q value a little based on LEARNING_RATE

        # Train the neural network with our updated Q value
        self.model.fit(np.array(state), np.array(q_current), verbose=0)

In [ ]:


### Train ###
agent = DQNAgent(env, learning_rate=0.001)

# Was stable after about 100 episodes
EPISODES = 200
for episode in range(EPISODES):
    state = env.reset()
    done = False
    cumulative_reward = 0
    i = 0
    env_i = 0
    while not done:
        action = agent.get_action(state)                                 # Action = get action from Agent
        next_state, reward, done, info = env.step(action)                # Do the Action
        cumulative_reward += reward

        if done and env.reset():
            reward = -100  # Punish for crashing

        agent.add_experience([state, action, next_state, reward, done])  # Add experience to replay buffer
        agent.train_network()                                            # Train the network

        i += 1
        state = next_state                                               # Set State to next State

        # env.render()
        # sleep(0.05)

    # Decay epsilon each episode
    agent.epsilon = max(agent.min_epsilon, agent.epsilon * agent.epsilon_decay)

    print(f"e: {episode:03}, eps: {agent.epsilon:.2f}, r: {cumulative_reward}")

In [ ]:

### Test run ###
print()
while True:
    obs = env.reset()
    done = False
    while not done:
        action = agent.get_action(state, skip_random=True)              # Action = get action from Agent
        state, reward, done, info = env.step(action)  # Do the Action
        env.render()
        sleep(0.5)
